In [2]:
pip install rouge

In [9]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from rouge import Rouge

# hyperparameters
batch_size = 64 # how many independent sequences
block_size = 256 # what is the maximum context
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------
torch.manual_seed(1337)
with open('/content/input (1).txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

def save_checkpoint(model, optimizer, iter, filepath):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'iteration': iter
    }
    torch.save(checkpoint, filepath)
    print(f"Model checkpoint saved at iteration {iter}.")

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        perplexities = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss, perplexity = model(X, Y)
            losses[k] = loss.item()
            perplexities[k] = perplexity.item() if perplexity is not None else float('nan')
        out[f'{split}_loss'] = losses.mean()
        out[f'{split}_perplexity'] = perplexities.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k =self.key(x)   # (B,T,C)
        q =self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei =q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei =wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei =F.softmax(wei, dim=-1) # (B, T, T)
        wei =self.dropout(wei)
        # perform the weighted aggregation of the values
        v =self.value(x) # (B,T,C)
        out =wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out =torch.cat([h(x) for h in self.heads], dim=-1)
        out =self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net =nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size =n_embd // n_head
        self.sa =MultiHeadAttention(n_head, head_size)
        self.ffwd =FeedFoward(n_embd)
        self.ln1 =nn.LayerNorm(n_embd)
        self.ln2 =nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x +self.sa(self.ln1(x))
        x = x +self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb =self.token_embedding_table(idx) # (B,T,C)
        pos_emb =self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x =tok_emb + pos_emb # (B,T,C)
        x =self.blocks(x) # (B,T,C)
        x =self.ln_f(x) # (B,T,C)
        logits =self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss=None
            perplexity=None
        else:
            B,T,C =logits.shape
            logits=logits.view(B*T, C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits, targets)
            probabilities=F.softmax(logits, dim=-1)
            log_probabilities=torch.log(probabilities)
            log_prob=torch.gather(log_probabilities,1,targets.unsqueeze(1)).squeeze(1)
            N=targets.size(0)
            perplexity=torch.exp(-torch.sum(log_prob)/N)

        return logits, loss,perplexity

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond =idx[:, -block_size:]
            # get the predictions
            logits, loss,perplexity = self(idx_cond)
            # focus only on the last time step
            logits =logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs =F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next =torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx =torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')


optimizer =torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):


    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train_loss']:.4f}, val loss {losses['val_loss']:.4f}, PERPLEX_TRAIN {losses['train_perplexity']:.4f} "+
              f"PERPLEX_VAL {losses['val_perplexity']:.4f}"
        )
        save_checkpoint(model, optimizer, iter, f"SUPER_BIGRAM_ITER_{iter}.pt")

    # sample a batch of data
    xb, yb =get_batch('train')

    # evaluate the loss
    logits,loss,perplexity = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context =torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))
print("#########-----------------------------------------------------------------------------------------------------------------------------###### \n \n")
rouge = Rouge()
generated_summaries = []
context = torch.tensor([[18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10]], dtype=torch.long, device=device)
summary= decode(m.generate(context, max_new_tokens=2000)[0].tolist())
generated_summaries.append(summary)
reference_summaries = []
reference_summaries.append(text[:2000])
print("REFERENCE SUMMARY: \n"+str(reference_summari es[0][:50])+"\n")
print("GENERATED SUMMARY: \n"+str(generated_summaries[0][:50])+"\n")
print("ROUGE_SCORES: \n")
rouge_scores = rouge.get_scores(generated_summaries, reference_summaries)
for score in rouge_scores:
    print(score)

10.788929 M parameters
step 0: train loss 4.2849, val loss 4.2823, PERPLEX_TRAIN 72.5948 PERPLEX_VAL 72.4087
Model checkpoint saved at iteration 0.
step 100: train loss 2.4585, val loss 2.4788, PERPLEX_TRAIN 11.6880 PERPLEX_VAL 11.9286
Model checkpoint saved at iteration 100.
step 200: train loss 2.3426, val loss 2.3912, PERPLEX_TRAIN 10.4093 PERPLEX_VAL 10.9273
Model checkpoint saved at iteration 200.
step 300: train loss 2.0551, val loss 2.1426, PERPLEX_TRAIN 7.8084 PERPLEX_VAL 8.5231
Model checkpoint saved at iteration 300.
step 400: train loss 1.8359, val loss 1.9604, PERPLEX_TRAIN 6.2720 PERPLEX_VAL 7.1033
Model checkpoint saved at iteration 400.
step 500: train loss 1.6761, val loss 1.8353, PERPLEX_TRAIN 5.3456 PERPLEX_VAL 6.2683
Model checkpoint saved at iteration 500.
step 600: train loss 1.5668, val loss 1.7468, PERPLEX_TRAIN 4.7921 PERPLEX_VAL 5.7379
Model checkpoint saved at iteration 600.
step 700: train loss 1.4915, val loss 1.6832, PERPLEX_TRAIN 4.4445 PERPLEX_VAL 5.3847


In [8]:
data[:14]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10])